In [3]:
import numpy as np
from scipy.optimize import linprog

def is_integer(x, eps):
    return abs(x - round(x)) <= eps

def solve(A, b, c, lb, ub, eps):
    tasks_solved = 0
    stack = [(A, b, c, lb, ub)]
    optimal_solution = None
    fun_value = -np.inf

    while True:
        try:
            A, b, c, lb, ub = stack.pop()
        except IndexError:
            break
        
        solution = linprog(
            c=-c,
            A_ub=A,
            b_ub=b,
            bounds=list(zip(lb, ub)),
            method='simplex'
        )
        tasks_solved += 1

        if solution.success:
            solution_x = solution.x
            solution_sum = -solution.fun

            if all(list(map(lambda x: is_integer(x, eps), solution_x))) and solution_sum > fun_value:
                optimal_solution = solution_x
                fun_value = solution_sum
            else:
                for i, component in enumerate(solution_x):
                    if not is_integer(component, eps) and solution_sum > fun_value:
                        modified_lb = np.copy(lb)
                        modified_ub = np.copy(ub)
                        modified_ub[i] = int(component)
                        modified_lb[i] = int(component) + 1

                        stack.append((A, b, c, lb, modified_ub))
                        stack.append((A, b, c, modified_lb, ub))


    if optimal_solution is None:
        print('No solution')

    print('Optimal solution: ', optimal_solution)
    print('Sum: ', fun_value)
    print('Tasks solved: ', tasks_solved)


A = np.array([
    [4, 3],
    [-4, 3]
])
b = np.array([22, 2])
c = np.array([-5, 4])
lb = np.array([1, 0])
ub = np.array([4, 5])

# A = np.array([
#     [5, 4, -2],
#     [5, -2, 4]
# ])
# b = np.array([6, 6])
# c = np.array([20, 2, 2])
# lb = np.array([0, 0, 0])
# ub = np.array([4, 4, 4])

solve(A, b, c, lb, ub, 10 ** (-10))


Optimal solution:  [1. 2.]
Sum:  3.0
Tasks solved:  9


In [1]:
import numpy as np
from scipy.optimize import linprog

In [7]:
class Task:
    def __init__(self, A, b, c, lb, ub):
        self.A = A
        self.b = b
        self.c = c
        self.lb = lb
        self.ub = ub
        

In [ ]:
def branch_and_bound()

In [8]:
A = np.array([
    [4, 3],
    [-4, 3]
])
b = np.array([22, 2])
c = np.array([-5, 4])
lb = np.array([1, 0])
ub = np.array([4, 5])

main_task = Task(A, b, c, lb, ub)

In [ ]:
branch_and_bound()